В данном ноутбуке мы реализуем алгоритм поиска пятен на снимке Солнца. Подключим необходимые модули

In [ ]:
%load_ext autotime

import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image 
import os

time: 437 ms (started: 2021-05-18 13:27:30 +03:00)


Определим вспомогательные функции

In [ ]:
def normalize(path, size):
    img = cv2.imread(path, 0)
    retval = cv2.haveImageReader(path)
    norm_img = np.zeros((size, size))
    final_img = cv2.normalize(img, norm_img, 0, 255, cv2.NORM_MINMAX)
    if retval:
        return final_img
    else:
        print(path.split('/')[-1], 'IS EMPTY!')
        return 0

def dist(a, b):
    return ((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2) ** (0.5)



time: 0 ns (started: 2021-05-18 13:27:31 +03:00)


Функция **findspots** определеяет на изображении пиксели, которые принадлежат области с пятном и возвращает массив этих пикселей, но предварительно фильтруя пиксели, соответсвующие одному пятну. Используя ее, функция **cutspots** вырезает с изображения квадратные снимки пятен в разрешении 256х256

In [ ]:
def findspots(path_to_image, flag = 1):
    
    square_size = 256 #size of spot square
    size = 4096 #size of image
    threshold = 133 #threshold parametr
    cropdist = 80 #min distance between 2 spots
    radius = 1400 #radius of Sun
    start = int(size/2) - radius
    stop = int(size/2) + radius
    step = 5
    img = normalize(path_to_image, size)

    if type(img) is np.ndarray:
        ret, thresh = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY)
        #cv2_imshow(thresh)

        spot_pixels = []

        for x in range(start, stop, step):
            for y in range(start, stop, step):
                if (x - size/2) ** 2 + (y - size/2) ** 2 <= radius ** 2 and int(thresh[x, y]) == 0:
                    spot_pixels.append([x, y])

        spot_pixels_m = spot_pixels.copy()
            
        for i in range(len(spot_pixels) - 1):
            for j in range(i + 1, len(spot_pixels)):
                if dist(spot_pixels[i], spot_pixels[j]) < cropdist:
                    if spot_pixels[j] in spot_pixels_m:
                        spot_pixels_m.remove(spot_pixels[j])

        if flag == 0:          
            print('spotpixels: %d' % len(spot_pixels))
            print('spots: %d' % len(spot_pixels_m))
            print('Spot coordinates: ', spot_pixels_m)
                
            for spot in spot_pixels_m:
                draw_square_spot = cv2.rectangle(img, (int(spot[1]) + int(square_size/2), int(spot[0]) + int(square_size/2)), (int(spot[1]) + 1 - int(square_size/2), int(spot[0]) + 1 - int(square_size/2)), (122, 160, 255), 2)  
            cv2.imwrite('test.jpg', draw_square_spot)     
#             cv2.imshow('t', draw_square_spot)
#             cv2.waitKey(0) 
#             cv2.destroyAllWindows()
#             img11 = Image.fromarray(np.asarray(draw_square_spot), 'RGB')
#             img11.show()
        return spot_pixels_m
    else:
        return []

def cut_spots(path_to_image, spot_size, flag = 1):
    spots_coord = findspots(path_to_image)
    spots = []
    image = cv2.imread(path_to_image)
    for spot in spots_coord:
        #cv2_imshow(image[spot[0] - int(spot_size/2) + 1:spot[0] + int(spot_size/2), spot[1] - int(spot_size/2) + 1:spot[1] + int(spot_size/2)])
        #spots.append(image[spot[0] - int(spot_size/2) + 1:spot[0] + int(spot_size/2), spot[1] - int(spot_size/2) + 1:spot[1] + int(spot_size/2)])
        spots.append(image[spot[0] - int(spot_size/2):spot[0] + int(spot_size/2), spot[1] - int(spot_size/2):spot[1] + int(spot_size/2)])
        if flag == 0:
            cv2.imwrite('test_spot.jpg', spots[-1])
    return spots
    


time: 0 ns (started: 2021-05-18 13:27:31 +03:00)


Проверим, сколько получилсоь снимков за 5 лет, учитывая, что в некоторые дни снимков не имеется, эти файлы были удалены

In [ ]:

path_to_save = './imgs_full/'
for year in range(2011, 2016):
    if not os.path.exists(path_to_save + str(year)):
        os.makedirs(path_to_save + str(year))

path_y2011 = path_to_save + '/2011/'
path_y2012 = path_to_save + '/2012/'
path_y2013 = path_to_save + '/2013/'
path_y2014 = path_to_save + '/2014/'
path_y2015 = path_to_save + '/2015/'
fds1 = sorted(os.listdir(path_y2011))
fds2 = sorted(os.listdir(path_y2012))
fds3 = sorted(os.listdir(path_y2013))
fds4 = sorted(os.listdir(path_y2014))
fds5 = sorted(os.listdir(path_y2015))
print(len(fds1), len(fds2), len(fds3), len(fds4), len(fds5), len(fds1) + len(fds2) + len(fds3) + len(fds4) + len(fds5))

363 365 365 365 349 1807
time: 47 ms (started: 2021-05-18 13:27:37 +03:00)


In [ ]:
Данная функция записывает в массив **spots** все пятна, полученные со всех изображений в папке

In [ ]:
def ARspot(path_to_dir):
    fds = sorted(os.listdir(path_to_dir))
    spots = []
    i = 1
    for img in fds:
        path_to_im = path_to_dir + img
        if cut_spots(path_to_im, 256) != []:
            spots += cut_spots(path_to_im, 256)
        print(img, ' done ' , i, '/', len(fds), '= %.2f'% (100*i/len(fds)), '%')
        i += 1
    return spots



time: 0 ns (started: 2021-05-18 13:27:39 +03:00)


In [ ]:
Далее последовательно вырезаются пятна в каждом году. Одновременный запуск на 5 годах выдавал ошибку по памяти

In [ ]:
spots1 = ARspot(path_y2011)

2011_01_01.jpg  done  1 / 363 = 0.28 %
2011_01_02.jpg  done  2 / 363 = 0.55 %
2011_01_03.jpg  done  3 / 363 = 0.83 %
2011_01_04.jpg  done  4 / 363 = 1.10 %
2011_01_05.jpg  done  5 / 363 = 1.38 %
2011_01_06.jpg  done  6 / 363 = 1.65 %
2011_01_07.jpg  done  7 / 363 = 1.93 %
2011_01_08.jpg  done  8 / 363 = 2.20 %
2011_01_09.jpg  done  9 / 363 = 2.48 %
2011_01_10.jpg  done  10 / 363 = 2.75 %
2011_01_11.jpg  done  11 / 363 = 3.03 %
2011_01_12.jpg  done  12 / 363 = 3.31 %
2011_01_13.jpg  done  13 / 363 = 3.58 %
2011_01_14.jpg  done  14 / 363 = 3.86 %
2011_01_15.jpg  done  15 / 363 = 4.13 %
2011_01_16.jpg  done  16 / 363 = 4.41 %
2011_01_17.jpg  done  17 / 363 = 4.68 %
2011_01_18.jpg  done  18 / 363 = 4.96 %
2011_01_19.jpg  done  19 / 363 = 5.23 %
2011_01_20.jpg  done  20 / 363 = 5.51 %
2011_01_21.jpg  done  21 / 363 = 5.79 %
2011_01_22.jpg  done  22 / 363 = 6.06 %
2011_01_23.jpg  done  23 / 363 = 6.34 %
2011_01_24.jpg  done  24 / 363 = 6.61 %
2011_01_25.jpg  done  25 / 363 = 6.89 %
2011_01_2

In [ ]:
print(len(spots1))

1792
time: 0 ns (started: 2021-05-18 12:52:35 +03:00)


In [ ]:
year = '2011'
path = './spots/spots_' + year + '/'
if not os.path.exists(path):
    os.makedirs(path)
for num, spot in enumerate(spots1):
    num += 1
    name = year + '_' + str(num) + '.jpg'
    im = Image.fromarray(spot)
    im.save(path + name)
    print(num, 'Done: %.2f' % (100 * num / len(spots1)))

1 Done: 0.06
2 Done: 0.11
3 Done: 0.17
4 Done: 0.22
5 Done: 0.28
6 Done: 0.33
7 Done: 0.39
8 Done: 0.45
9 Done: 0.50
10 Done: 0.56
11 Done: 0.61
12 Done: 0.67
13 Done: 0.73
14 Done: 0.78
15 Done: 0.84
16 Done: 0.89
17 Done: 0.95
18 Done: 1.00
19 Done: 1.06
20 Done: 1.12
21 Done: 1.17
22 Done: 1.23
23 Done: 1.28
24 Done: 1.34
25 Done: 1.40
26 Done: 1.45
27 Done: 1.51
28 Done: 1.56
29 Done: 1.62
30 Done: 1.67
31 Done: 1.73
32 Done: 1.79
33 Done: 1.84
34 Done: 1.90
35 Done: 1.95
36 Done: 2.01
37 Done: 2.06
38 Done: 2.12
39 Done: 2.18
40 Done: 2.23
41 Done: 2.29
42 Done: 2.34
43 Done: 2.40
44 Done: 2.46
45 Done: 2.51
46 Done: 2.57
47 Done: 2.62
48 Done: 2.68
49 Done: 2.73
50 Done: 2.79
51 Done: 2.85
52 Done: 2.90
53 Done: 2.96
54 Done: 3.01
55 Done: 3.07
56 Done: 3.12
57 Done: 3.18
58 Done: 3.24
59 Done: 3.29
60 Done: 3.35
61 Done: 3.40
62 Done: 3.46
63 Done: 3.52
64 Done: 3.57
65 Done: 3.63
66 Done: 3.68
67 Done: 3.74
68 Done: 3.79
69 Done: 3.85
70 Done: 3.91
71 Done: 3.96
72 Done: 4.02
7

In [ ]:
spots2 = ARspot(path_y2012)
print(len(spots2))

2012_01_01.jpg  done  1 / 365 = 0.27 %
2012_01_02.jpg  done  2 / 365 = 0.55 %
2012_01_03.jpg  done  3 / 365 = 0.82 %
2012_01_04.jpg  done  4 / 365 = 1.10 %
2012_01_05.jpg  done  5 / 365 = 1.37 %
2012_01_06.jpg  done  6 / 365 = 1.64 %
2012_01_07.jpg  done  7 / 365 = 1.92 %
2012_01_08.jpg  done  8 / 365 = 2.19 %
2012_01_09.jpg  done  9 / 365 = 2.47 %
2012_01_10.jpg  done  10 / 365 = 2.74 %
2012_01_11.jpg  done  11 / 365 = 3.01 %
2012_01_12.jpg  done  12 / 365 = 3.29 %
2012_01_13.jpg  done  13 / 365 = 3.56 %
2012_01_14.jpg  done  14 / 365 = 3.84 %
2012_01_15.jpg  done  15 / 365 = 4.11 %
2012_01_16.jpg  done  16 / 365 = 4.38 %
2012_01_17.jpg  done  17 / 365 = 4.66 %
2012_01_18.jpg  done  18 / 365 = 4.93 %
2012_01_19.jpg  done  19 / 365 = 5.21 %
2012_01_20.jpg  done  20 / 365 = 5.48 %
2012_01_21.jpg  done  21 / 365 = 5.75 %
2012_01_22.jpg  done  22 / 365 = 6.03 %
2012_01_23.jpg  done  23 / 365 = 6.30 %
2012_01_24.jpg  done  24 / 365 = 6.58 %
2012_01_25.jpg  done  25 / 365 = 6.85 %
2012_01_2

In [ ]:
year = '2012'
path = './spots/spots_' + year + '/'
if not os.path.exists(path):
    os.makedirs(path)
for num, spot in enumerate(spots2):
    num += 1
    name = year + '_' + str(num) + '.jpg'
    im = Image.fromarray(spot)
    im.save(path + name)
    print(num, 'Done: %.2f' % (100 * num / len(spots2)))

1 Done: 0.05
2 Done: 0.11
3 Done: 0.16
4 Done: 0.21
5 Done: 0.27
6 Done: 0.32
7 Done: 0.37
8 Done: 0.43
9 Done: 0.48
10 Done: 0.53
11 Done: 0.59
12 Done: 0.64
13 Done: 0.69
14 Done: 0.75
15 Done: 0.80
16 Done: 0.85
17 Done: 0.91
18 Done: 0.96
19 Done: 1.01
20 Done: 1.07
21 Done: 1.12
22 Done: 1.17
23 Done: 1.23
24 Done: 1.28
25 Done: 1.33
26 Done: 1.39
27 Done: 1.44
28 Done: 1.49
29 Done: 1.55
30 Done: 1.60
31 Done: 1.65
32 Done: 1.71
33 Done: 1.76
34 Done: 1.81
35 Done: 1.87
36 Done: 1.92
37 Done: 1.97
38 Done: 2.03
39 Done: 2.08
40 Done: 2.13
41 Done: 2.19
42 Done: 2.24
43 Done: 2.29
44 Done: 2.35
45 Done: 2.40
46 Done: 2.45
47 Done: 2.51
48 Done: 2.56
49 Done: 2.61
50 Done: 2.67
51 Done: 2.72
52 Done: 2.77
53 Done: 2.83
54 Done: 2.88
55 Done: 2.93
56 Done: 2.99
57 Done: 3.04
58 Done: 3.09
59 Done: 3.15
60 Done: 3.20
61 Done: 3.26
62 Done: 3.31
63 Done: 3.36
64 Done: 3.42
65 Done: 3.47
66 Done: 3.52
67 Done: 3.58
68 Done: 3.63
69 Done: 3.68
70 Done: 3.74
71 Done: 3.79
72 Done: 3.84
7

In [ ]:
spots3 = ARspot(path_y2013)
print(len(spots3))

2013_01_01.jpg  done  1 / 365 = 0.27 %
2013_01_02.jpg  done  2 / 365 = 0.55 %
2013_01_03.jpg  done  3 / 365 = 0.82 %
2013_01_04.jpg  done  4 / 365 = 1.10 %
2013_01_05.jpg  done  5 / 365 = 1.37 %
2013_01_06.jpg  done  6 / 365 = 1.64 %
2013_01_07.jpg  done  7 / 365 = 1.92 %
2013_01_08.jpg  done  8 / 365 = 2.19 %
2013_01_09.jpg  done  9 / 365 = 2.47 %
2013_01_10.jpg  done  10 / 365 = 2.74 %
2013_01_11.jpg  done  11 / 365 = 3.01 %
2013_01_12.jpg  done  12 / 365 = 3.29 %
2013_01_13.jpg  done  13 / 365 = 3.56 %
2013_01_14.jpg  done  14 / 365 = 3.84 %
2013_01_15.jpg  done  15 / 365 = 4.11 %
2013_01_16.jpg  done  16 / 365 = 4.38 %
2013_01_17.jpg  done  17 / 365 = 4.66 %
2013_01_18.jpg  done  18 / 365 = 4.93 %
2013_01_19.jpg  done  19 / 365 = 5.21 %
2013_01_20.jpg  done  20 / 365 = 5.48 %
2013_01_21.jpg  done  21 / 365 = 5.75 %
2013_01_22.jpg  done  22 / 365 = 6.03 %
2013_01_23.jpg  done  23 / 365 = 6.30 %
2013_01_24.jpg  done  24 / 365 = 6.58 %
2013_01_25.jpg  done  25 / 365 = 6.85 %
2013_01_2

In [ ]:
year = '2013'
path = './spots/spots_' + year + '/'
if not os.path.exists(path):
    os.makedirs(path)
for num, spot in enumerate(spots3):
    num += 1
    name = year + '_' + str(num) + '.jpg'
    im = Image.fromarray(spot)
    im.save(path + name)
    print(num, 'Done: %.2f' % (100 * num / len(spots3)))

1 Done: 0.05
2 Done: 0.10
3 Done: 0.14
4 Done: 0.19
5 Done: 0.24
6 Done: 0.29
7 Done: 0.34
8 Done: 0.38
9 Done: 0.43
10 Done: 0.48
11 Done: 0.53
12 Done: 0.58
13 Done: 0.62
14 Done: 0.67
15 Done: 0.72
16 Done: 0.77
17 Done: 0.81
18 Done: 0.86
19 Done: 0.91
20 Done: 0.96
21 Done: 1.01
22 Done: 1.05
23 Done: 1.10
24 Done: 1.15
25 Done: 1.20
26 Done: 1.25
27 Done: 1.29
28 Done: 1.34
29 Done: 1.39
30 Done: 1.44
31 Done: 1.49
32 Done: 1.53
33 Done: 1.58
34 Done: 1.63
35 Done: 1.68
36 Done: 1.73
37 Done: 1.77
38 Done: 1.82
39 Done: 1.87
40 Done: 1.92
41 Done: 1.97
42 Done: 2.01
43 Done: 2.06
44 Done: 2.11
45 Done: 2.16
46 Done: 2.21
47 Done: 2.25
48 Done: 2.30
49 Done: 2.35
50 Done: 2.40
51 Done: 2.44
52 Done: 2.49
53 Done: 2.54
54 Done: 2.59
55 Done: 2.64
56 Done: 2.68
57 Done: 2.73
58 Done: 2.78
59 Done: 2.83
60 Done: 2.88
61 Done: 2.92
62 Done: 2.97
63 Done: 3.02
64 Done: 3.07
65 Done: 3.12
66 Done: 3.16
67 Done: 3.21
68 Done: 3.26
69 Done: 3.31
70 Done: 3.36
71 Done: 3.40
72 Done: 3.45
7

In [ ]:
spots4 = ARspot(path_y2014)
print(len(spots4))

2014_01_01.jpg  done  1 / 365 = 0.27 %
2014_01_02.jpg  done  2 / 365 = 0.55 %
2014_01_03.jpg  done  3 / 365 = 0.82 %
2014_01_04.jpg  done  4 / 365 = 1.10 %
2014_01_05.jpg  done  5 / 365 = 1.37 %
2014_01_06.jpg  done  6 / 365 = 1.64 %
2014_01_07.jpg  done  7 / 365 = 1.92 %
2014_01_08.jpg  done  8 / 365 = 2.19 %
2014_01_09.jpg  done  9 / 365 = 2.47 %
2014_01_10.jpg  done  10 / 365 = 2.74 %
2014_01_11.jpg  done  11 / 365 = 3.01 %
2014_01_12.jpg  done  12 / 365 = 3.29 %
2014_01_13.jpg  done  13 / 365 = 3.56 %
2014_01_14.jpg  done  14 / 365 = 3.84 %
2014_01_15.jpg  done  15 / 365 = 4.11 %
2014_01_16.jpg  done  16 / 365 = 4.38 %
2014_01_17.jpg  done  17 / 365 = 4.66 %
2014_01_18.jpg  done  18 / 365 = 4.93 %
2014_01_19.jpg  done  19 / 365 = 5.21 %
2014_01_20.jpg  done  20 / 365 = 5.48 %
2014_01_21.jpg  done  21 / 365 = 5.75 %
2014_01_22.jpg  done  22 / 365 = 6.03 %
2014_01_23.jpg  done  23 / 365 = 6.30 %
2014_01_24.jpg  done  24 / 365 = 6.58 %
2014_01_25.jpg  done  25 / 365 = 6.85 %
2014_01_2

In [ ]:
year = '2014'
path = './spots/spots_' + year + '/'
if not os.path.exists(path):
    os.makedirs(path)
for num, spot in enumerate(spots4):
    num += 1
    name = year + '_' + str(num) + '.jpg'
    im = Image.fromarray(spot)
    im.save(path + name)
    print(num, 'Done: %.2f' % (100 * num / len(spots4)))

1 Done: 0.04
2 Done: 0.08
3 Done: 0.12
4 Done: 0.16
5 Done: 0.20
6 Done: 0.24
7 Done: 0.28
8 Done: 0.31
9 Done: 0.35
10 Done: 0.39
11 Done: 0.43
12 Done: 0.47
13 Done: 0.51
14 Done: 0.55
15 Done: 0.59
16 Done: 0.63
17 Done: 0.67
18 Done: 0.71
19 Done: 0.75
20 Done: 0.79
21 Done: 0.83
22 Done: 0.87
23 Done: 0.91
24 Done: 0.94
25 Done: 0.98
26 Done: 1.02
27 Done: 1.06
28 Done: 1.10
29 Done: 1.14
30 Done: 1.18
31 Done: 1.22
32 Done: 1.26
33 Done: 1.30
34 Done: 1.34
35 Done: 1.38
36 Done: 1.42
37 Done: 1.46
38 Done: 1.50
39 Done: 1.54
40 Done: 1.57
41 Done: 1.61
42 Done: 1.65
43 Done: 1.69
44 Done: 1.73
45 Done: 1.77
46 Done: 1.81
47 Done: 1.85
48 Done: 1.89
49 Done: 1.93
50 Done: 1.97
51 Done: 2.01
52 Done: 2.05
53 Done: 2.09
54 Done: 2.13
55 Done: 2.17
56 Done: 2.20
57 Done: 2.24
58 Done: 2.28
59 Done: 2.32
60 Done: 2.36
61 Done: 2.40
62 Done: 2.44
63 Done: 2.48
64 Done: 2.52
65 Done: 2.56
66 Done: 2.60
67 Done: 2.64
68 Done: 2.68
69 Done: 2.72
70 Done: 2.76
71 Done: 2.80
72 Done: 2.83
7

In [ ]:
spots5 = ARspot(path_y2015)
print(len(spots5))

2015_01_01.jpg  done  1 / 349 = 0.29 %
2015_01_02.jpg  done  2 / 349 = 0.57 %
2015_01_03.jpg  done  3 / 349 = 0.86 %
2015_01_04.jpg  done  4 / 349 = 1.15 %
2015_01_05.jpg  done  5 / 349 = 1.43 %
2015_01_06.jpg  done  6 / 349 = 1.72 %
2015_01_07.jpg  done  7 / 349 = 2.01 %
2015_01_08.jpg  done  8 / 349 = 2.29 %
2015_01_09.jpg  done  9 / 349 = 2.58 %
2015_01_10.jpg  done  10 / 349 = 2.87 %
2015_01_11.jpg  done  11 / 349 = 3.15 %
2015_01_12.jpg  done  12 / 349 = 3.44 %
2015_01_13.jpg  done  13 / 349 = 3.72 %
2015_01_14.jpg  done  14 / 349 = 4.01 %
2015_01_15.jpg  done  15 / 349 = 4.30 %
2015_01_16.jpg  done  16 / 349 = 4.58 %
2015_01_17.jpg  done  17 / 349 = 4.87 %
2015_01_18.jpg  done  18 / 349 = 5.16 %
2015_01_19.jpg  done  19 / 349 = 5.44 %
2015_01_20.jpg  done  20 / 349 = 5.73 %
2015_01_21.jpg  done  21 / 349 = 6.02 %
2015_01_22.jpg  done  22 / 349 = 6.30 %
2015_01_23.jpg  done  23 / 349 = 6.59 %
2015_01_24.jpg  done  24 / 349 = 6.88 %
2015_01_25.jpg  done  25 / 349 = 7.16 %
2015_01_2

In [ ]:
year = '2015'
path = './spots/spots_' + year + '/'
if not os.path.exists(path):
    os.makedirs(path)
for num, spot in enumerate(spots5):
    num += 1
    name = year + '_' + str(num) + '.jpg'
    im = Image.fromarray(spot)
    im.save(path + name)
    print(num, 'Done: %.2f' % (100 * num / len(spots5)))

1 Done: 0.07
2 Done: 0.14
3 Done: 0.21
4 Done: 0.28
5 Done: 0.35
6 Done: 0.42
7 Done: 0.49
8 Done: 0.56
9 Done: 0.63
10 Done: 0.70
11 Done: 0.77
12 Done: 0.84
13 Done: 0.91
14 Done: 0.97
15 Done: 1.04
16 Done: 1.11
17 Done: 1.18
18 Done: 1.25
19 Done: 1.32
20 Done: 1.39
21 Done: 1.46
22 Done: 1.53
23 Done: 1.60
24 Done: 1.67
25 Done: 1.74
26 Done: 1.81
27 Done: 1.88
28 Done: 1.95
29 Done: 2.02
30 Done: 2.09
31 Done: 2.16
32 Done: 2.23
33 Done: 2.30
34 Done: 2.37
35 Done: 2.44
36 Done: 2.51
37 Done: 2.58
38 Done: 2.65
39 Done: 2.72
40 Done: 2.79
41 Done: 2.86
42 Done: 2.92
43 Done: 2.99
44 Done: 3.06
45 Done: 3.13
46 Done: 3.20
47 Done: 3.27
48 Done: 3.34
49 Done: 3.41
50 Done: 3.48
51 Done: 3.55
52 Done: 3.62
53 Done: 3.69
54 Done: 3.76
55 Done: 3.83
56 Done: 3.90
57 Done: 3.97
58 Done: 4.04
59 Done: 4.11
60 Done: 4.18
61 Done: 4.25
62 Done: 4.32
63 Done: 4.39
64 Done: 4.46
65 Done: 4.53
66 Done: 4.60
67 Done: 4.67
68 Done: 4.74
69 Done: 4.81
70 Done: 4.87
71 Done: 4.94
72 Done: 5.01
7

In [ ]:
path_s1 = './spots/spots_2011'
path_s2 = './spots/spots_2012'
path_s3 = './spots/spots_2013'
path_s4 = './spots/spots_2014'
path_s5 = './spots/spots_2015'
fdss1 = sorted(os.listdir(path_s1))
fdss2 = sorted(os.listdir(path_s2))
fdss3 = sorted(os.listdir(path_s3))
fdss4 = sorted(os.listdir(path_s4))
fdss5 = sorted(os.listdir(path_s5))
print(len(fdss1), len(fdss2), len(fdss3), len(fdss4), len(fdss5), len(fdss1) + len(fdss2) + len(fdss3) + len(fdss4) + len(fdss5))

1792 1874 2086 2540 1436 9728
time: 109 ms (started: 2021-05-18 13:42:55 +03:00)
